In [1]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,34 mins 46 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.4
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_malbalawi_0euhae
H2O cluster total nodes:,1
H2O cluster free memory:,1.641 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [13]:
train = h2o.get_frame("Pepole_Train")
valid = h2o.get_frame("Pepole_Valdit")
test= h2o.get_frame("Pepole_Test")

In [14]:
print("%d/%d/%d" % (train.nrows,valid.nrows,test.nrows))

788/118/94


In [10]:
# Bring the GBM 
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [15]:
# Start learn the Model: Here we want to exclude the incomeand id cloumns 
y="income"
ignoreFields=[y,"id"]
x= [i for i in train.names if i not in ignoreFields]

In [16]:
# specify model id and validation frame 
m1= H2OGradientBoostingEstimator(model_id="defaults")
m1.train(x,y,train,
        validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [18]:
# we want just check how the model did in the three dateset Train  Valid,Test and check the mean arithmetic error of the model 
m1.mae(train=True)

884.0323356202412

In [19]:
m1.mae(valid=True)

1465.606832714099

In [22]:
perf=m1.model_performance(test)
perf.mae()

1441.367701394246

# Creat an Overfit A GBM by increase the n trees and max depth for the Model: we will compare the two model by theh ave error in the TEST and Valid dateset 

In [23]:
# specify model id and validation frame the defult for the n tree is 50 and the max depth is 5 but here we increased 
m2= H2OGradientBoostingEstimator(model_id="Overfit",ntrees= 1000,
        max_depth=10)
m2.train(x,y,train,
        validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [47]:
print(" Model1--> Model2")
print("Train:  %d -->%d"    % (m1.mae(train=True),m2.mae(train=True) ))
print("Valid:  %d -->%d"    %(m1.mae(valid=True),m2.mae(valid=True)))
print("Test:   %d -->%d"    %(perf.mae(),m2.model_performance(test).mae()))

 Model1--> Model2
Train:  884 -->200
Valid:  1465 -->1577
Test:   1441 -->1570


# The training error have really come down. But validation error, test error have gone up. This is bad. We want, we don't care about the training error. Training error never matters. 
# What matters is the error on test and we've made it 200-worth by giving it more trees and overfiting